In [1]:
import os
from pathlib import Path
import torch
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig, SelectionDiscipline
from pygpudrive.env.env_torch import GPUDriveTorchEnv

In [2]:
MAX_NUM_OBJECTS = 32  # Maximum number of objects in the scene we control
NUM_WORLDS = 4  # Number of parallel environments
K_UNIQUE_SCENES = 1 # Number of unique scenes

# Set the path to where you want to save the videos
VIDEO_PATH = "./videos"

FPS = 4  # Video frames per second

device = 'cpu' # for explanation purposes in notebook #torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
scene_config = SceneConfig(
    path="data/processed/f1_saudi", 
    num_scenes=NUM_WORLDS,
    discipline=SelectionDiscipline.K_UNIQUE_N,
    k_unique_scenes=K_UNIQUE_SCENES,
)

In [4]:
env_config = EnvConfig(
    steer_actions = torch.round(
        torch.linspace(-1.0, 1.0, 3), decimals=3),
    accel_actions = torch.round(
        torch.linspace(-3, 3, 3), decimals=3
    )
)

In [5]:
render_config = RenderConfig(
    resolution=(1600, 1600), # Quality of the rendered images
    draw_obj_idx=True, # Draw object index on the objects
)

In [6]:
# MAKE ENV
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    render_config=render_config,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device=device,
)


--- Ratio unique scenes / number of worls = 1 / 4 ---



Invalid MIT-MAGIC-COOKIE-1 key


In [7]:
obs = env.reset()
frames = {f"env_{i}": [] for i in range(NUM_WORLDS)}

for _ in range(env_config.episode_len):
    # SELECT ACTIONS
    rand_action = torch.Tensor(
        [[env.action_space.sample() for _ in range(MAX_NUM_OBJECTS * NUM_WORLDS)]]
    ).reshape(NUM_WORLDS, MAX_NUM_OBJECTS)

    #print(rand_action.shape)
    # STEP
    env.step_dynamics(rand_action)

    obs = env.get_obs()
    reward = env.get_rewards()
    done = env.get_dones()

    # RENDER
    for i in range(NUM_WORLDS):
        frames[f"env_{i}"].append(env.render(i))

In [8]:
# Show videos
mediapy.show_videos(frames, fps=30, width=750, height=500, columns=2)

env_0 This browser does not support the video tag.,env_1 This browser does not support the video tag.
env_2 This browser does not support the video tag.,env_3 This browser does not support the video tag.
